# Examples

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Error-in-Cylinder" data-toc-modified-id="Error-in-Cylinder-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Error in Cylinder</a></span></li><li><span><a href="#Error-in-Tapered-Cylinder" data-toc-modified-id="Error-in-Tapered-Cylinder-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Error in Tapered Cylinder</a></span></li></ul></div>

### Error in Cylinder

The below example determines the tolerances for the cross sectional area $(A)$ and volume $(V)$ of a cylinder given the measurement tolerances for the radius $(r)$ and height $(h)$.

In [1]:
from math import pi
from error_solver import ErrorSolver

# Variables     Value         Tolerance
r, rtol     =   5,            0.05
h, htol     =   12,           0.05
A, Atol     =   pi * r**2,    None
V, Vtol     =   A * h,        None

# List of equations
equations = ['A = pi * r**2',
             'V = A * h']

# Dictionaries of variables
values = {'h': h,
          'r': r,
          'A': A,
          'V': V}

errors = {'h': rtol,
          'r': htol}

solver = ErrorSolver(equations, values, errors)
solution = solver.solve()

print(solution['summary'])

 Variable | Value | Error Tolerance | Percent Error | Unknown 
    h     |  12   |      0.05       |     0.42      |  False  
    r     |   5   |      0.05       |     1.00      |  False  
    A     | 78.54 |      1.571      |     2.00      |  True   
    V     | 942.5 |      22.78      |     2.42      |  True   


### Error in Tapered Cylinder

The below example determines the tolerances for the cross sectional areas $(A_1, A_2)$ and volume $(V)$ of a cylinder given the measurement tolerances for the radii $(r_1, r_2)$ and height $(h)$.

In [2]:
from math import pi
from error_solver import ErrorSolver

# Variables     Value             Tolerance
h, htol     =   12,               0.05
r1, r1tol   =   2,                0.05
r2, r2tol   =   5,                0.05
A1, A1tol   =   pi * r1**2,       None
A2, A2tol   =   pi * r2**2,       None
V, Vtol     =   0.5*(A1 + A2)*h,  None

# List of equations
equations = ['A1 = pi * r1**2',
             'A2 = pi * r2**2',
             'V = 0.5*(A1 + A2)*h']

# Dictionaries of variables
values = {'h' : h,
          'r1': r1,
          'r2': r2,
          'A1': A1,
          'A2': A2,
          'V' : V}

errors = {'h': htol,
          'r1': r1tol,
          'r2': r2tol}

solver = ErrorSolver(equations, values, errors)
solution = solver.solve()

print(solution['summary'])

 Variable | Value | Error Tolerance | Percent Error | Unknown 
    h     |  12   |      0.05       |     0.42      |  False  
    r1    |   2   |      0.05       |     2.50      |  False  
    r2    |   5   |      0.05       |     1.00      |  False  
    A1    | 12.57 |     0.6283      |     5.00      |  True   
    A2    | 78.54 |      1.571      |     2.00      |  True   
    V     | 546.6 |      15.47      |     2.83      |  True   
